In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Webscraping

In [3]:

import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

df = pd.DataFrame(columns=["date", "text", "link"])
debatelink = []

base_url = "https://www.debates.org"
page = requests.get(urljoin(base_url, "/voter-education/debate-transcripts/"))
soup = BeautifulSoup(page.content, 'html.parser')

links = soup.find_all("a")
for link in links:
    if "debate-transcript" in link.get("href") and "Media" not in link.get("href"):
        debatelink.append(urljoin(base_url, link.get("href")))
        df.loc[len(df)] = [None, None, urljoin(base_url, link.get("href"))]

for link in df["link"]:
    try:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        text = soup.find_all("p")
        text = [t.get_text() for t in text]
        text = " ".join(text)
        df.loc[df["link"] == link, "text"] = text

        date_link = link.split("/")[5:6] 
        date_link = "-".join(date_link[0].split("-")[:3])
        df.loc[df["link"] == link, "date"] = date_link
    except requests.exceptions.RequestException as e:
        print(f"Error accessing {link}: {e}")

df


,date,text,link
0,,Unofficial transcripts of most presidential an...,https://www.debates.org/voter-education/debate...
1,vice-presidential-debate,Vice Presidential Debate at the University of ...,https://www.debates.org/voter-education/debate...
2,october-22-2020,Presidential Debate at Belmont University in N...,https://www.debates.org/voter-education/debate...
3,september-29-2020,Presidential Debate at Case Western Reserve Un...,https://www.debates.org/voter-education/debate...
4,october-19-2016,Presidential Debate at the University of Nevad...,https://www.debates.org/voter-education/debate...
...,...,...,...
92,october-22-1976,"\nOctober 22, 1976\n The Third Carter-Ford Pre...",https://www.debates.org/voter-education/debate...
93,september-26-1960,"\nSeptember 26, 1960\n The First Kennedy-Nixon...",https://www.debates.org/voter-education/debate...
94,october-7-1960,"\nOctober 7, 1960\n The Second Kennedy-Nixon P...",https://www.debates.org/voter-education/debate...
95,october-13-1960,"\nOctober 13, 1960\n The Third Kennedy-Nixon P...",https://www.debates.org/voter-education/debate...


In [4]:
df['text'][5]

'Presidential Debate at Washington University in St. Louis, Missouri October 9, 2016 PARTICIPANTS: Former Secretary of State Hillary Clinton (D) and Businessman Donald Trump (R) MODERATORS: Anderson Cooper (CNN) and  Martha Raddatz (ABC News) RADDATZ: Good evening. I’m Martha Raddatz from ABC News. COOPER: And I’m Anderson Cooper from CNN. We want to welcome you to Washington University in St. Louis for the second presidential debate between Hillary Clinton and Donald Trump, sponsored by the Commission on Presidential Debates. Tonight’s debate is a town hall format, which gives voters a chance to directly ask the candidates questions. Martha and I will ask follow-up questions but the night really belongs to the people in this room, and to people across the country who have submitted questions online.  RADDATZ: The people you see on this stage were chosen by the Gallup Organization. They are all from the St. Louis area and told Gallup they haven’t committed to a candidate. Each of them 

In [4]:
len(debatelink)

97

In [5]:
req = requests.get('https://www.presidency.ucsb.edu/documents/republican-candidates-debate-des-moines-iowa-1')

In [6]:
req = requests.get('https://www.presidency.ucsb.edu/documents/republican-candidates-debate-des-moines-iowa-1')

In [7]:
req.status_code

200

In [8]:
req.content

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"\n  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">\n<html lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema#">\n<head profile="http://www.w3.org/1999/xhtml/vocab">\n  <meta charset="utf-8"><script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"dee899de70",applicationID:"80106271"};;/*! For license information please see nr-loader-rum-1.250.0.min.js.LICENSE.txt */\n(()=>{var e,t,r={234:(e,t,r)=>{"use strict";r.d(t,{P_:()=>v,Mt:()=>b,C5:()=>s,DL:()=>A,OP:()=>S,lF:()=>T,Yu:()=>x,Dg:()=>m,CX:()=>c,GE:()=>_,sU:()=>R});var n=r(8632),i=r(

In [9]:
req.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"\n  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">\n<html lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema#">\n<head profile="http://www.w3.org/1999/xhtml/vocab">\n  <meta charset="utf-8"><script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"dee899de70",applicationID:"80106271"};;/*! For license information please see nr-loader-rum-1.250.0.min.js.LICENSE.txt */\n(()=>{var e,t,r={234:(e,t,r)=>{"use strict";r.d(t,{P_:()=>v,Mt:()=>b,C5:()=>s,DL:()=>A,OP:()=>S,lF:()=>T,Yu:()=>x,Dg:()=>m,CX:()=>c,GE:()=>_,sU:()=>R});var n=r(8632),i=r(9

In [10]:
sitesoup = BeautifulSoup(req.content)

In [ ]:
[tag.text for tag in sitesoup.select('p')]

['About\xa0Search',
 '',
 'PARTICIPANTS:Governor Ron DeSantis (FL); andFormer Governor Nikki Haley (SC);',
 'MODERATORS:Dana Bash (CNN); andJake Tapper (CNN)',
 'TAPPER: Live from Drake University in Des Moines, Iowa, this is the CNN Republican presidential debate.',
 'We want to welcome our viewers in the United States and around the world watching us on CNN, CNN Max, CNN International, CNN en Espanol, and CNN.com.',
 "Good evening. I'm Jake Tapper, moderating tonight's debate alongside my friend and colleague Dana Bash.",
 'BASH: And just five days from now, voters here in Iowa will cast the first votes of the 2024 presidential race.',
 "Tonight, the very first one-on-one matchup between top contenders Governor Ron DeSantis and former Governor and U.N. Ambassador Nikki Haley. Former President Trump met the requirements for tonight's debate, but declined to participate.",
 'TAPPER: Our commitment in this debate is to provide as much information as possible to voters, especially here i

In [9]:
import re

# Function to extract text between all-caps words followed by a colon
def extract_text_segments(text, name):
    pattern = f"{name}:\s*([\s\S]*?)(?=(?:[A-Z]+\s*:\s*|$))"
    matches = re.finditer(pattern, text)
    indices = [(match.start(), match.end()) for match in matches]

    # Debugging: print indices to see the matches
    print(f"Indices: {indices}")

    extracted_texts = []
    for i in range(len(indices) - 1):
        start = indices[i][0]
        end = indices[i][1]
        extracted_text = text[start:end].strip()
        extracted_texts.append(extracted_text)

    return extracted_texts

extract_text_segments(df['text'][5], 'CLINTON')

Indices: [(2177, 3952), (8946, 10940), (14304, 15837), (18552, 18639), (18658, 18698), (18757, 19330), (19986, 21127), (22854, 22911), (22984, 23032), (23057, 23103), (23220, 23257), (23297, 23484), (23518, 23721), (23793, 23888), (24660, 24727), (24812, 26754), (26798, 26883), (29260, 30366), (30393, 30541), (30566, 30807), (34705, 36521), (38988, 40558), (40591, 40612), (40647, 40703), (43073, 44823), (44905, 44960), (50240, 52157), (52187, 52339), (55331, 55505), (55586, 55648), (55667, 55747), (56021, 57679), (57709, 57798), (59121, 60884), (61035, 61114), (61170, 61233), (67136, 67622), (67696, 67763), (67782, 68824), (71603, 73382), (73420, 73555), (73587, 73657), (74150, 74879), (74898, 75018), (77909, 77922), (77966, 78871), (78902, 78948), (78988, 79027), (79072, 79260), (80003, 82222), (84142, 84168), (84242, 84494), (84571, 84630), (86921, 88869), (88896, 88990), (89015, 89136), (89540, 90872)]


['CLINTON: Well, thank you. Are you a teacher? Yes, I think that that’s a very good question, because I’ve heard from lots of teachers and parents about some of their concerns about some of the things that are being said and done in this campaign. And I think it is very important for us to make clear to our children that our country really is great because we’re good. And we are going to respect one another, lift each other up. We are going to be looking for ways to celebrate our diversity, and we are going to try to reach out to every boy and girl, as well as every adult, to bring them in to working on behalf of our country. I have a very positive and optimistic view about what we can do together. That’s why the slogan of my campaign is “Stronger Together,” because I think if we work together, if we overcome the divisiveness that sometimes sets Americans against one another, and instead we make some big goals—and I’ve set forth some big goals, getting the economy to work for everyone,

# NLP

In [ ]:
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')

from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import string
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, ConfusionMatrixDisplay

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,\
HashingVectorizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shelleywang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/shelleywang/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shelleywang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
tokenizer = RegexpTokenizer(r"(?u)\w{3,}")
stopwords = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def preprocessing(text, tokenizer, stopwords, lemmatizer):
    # Lower case
    text = text.lower()
    
    # Tokenize
    tokens = tokenizer.tokenize(text)
    
    # Remove stopwords
    tokens =  [token for token in tokens if token not in stopwords]
    
    # Lemmatize
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens

In [ ]:
tokenized_df = df['text'].apply((lambda x: preprocessing(x, tokenizer, stopwords, lemmatizer)))
df['tokens'] = tokenized_df

In [ ]:
df['string_tokens'] = df['tokens'].str.join(" ")

## NLP Modelling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df["string_tokens"], df['target'], random_state=42)

In [ ]:
vectorizer = TfidfVectorizer()
vectorizr.fit(X_train
X_train_vectorized = vectorizer.transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
mnb = MultinomialNB()
mnb = mnb.fit(X_train_vectorized, y_train)
